In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv', index_col='id')
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv', index_col='id')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

In [ ]:
X = train_data.drop(['claim'], axis=1)
y = train_data.claim

**Based on the notebook 'TabularPlaygroundSeriesDataAnalysis', we are going to drop for the moment some comlumns**

In [ ]:
to_beDropped=['f43', 'f49', 'f66', 'f72','f74', 'f88', 'f94','f101','f115']
X = X.drop(to_beDropped, axis=1)

In [ ]:
X.head()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

**impute missing values**

In [ ]:
imputer=SimpleImputer(strategy='median')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', imputer, list(set(X_train.columns)-set(to_beDropped)))
    ])

In [ ]:
model= XGBRegressor(
    n_estimators=10000,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.1,
    max_depth=3,
    booster='gbtree',
    reg_lambda=0.0008,
    reg_alpha=24,
    random_state=42
)

In [ ]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Transform the data
my_pipeline.fit(X_train, y_train) 
preds = model.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
preds = model.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)
print(mean_squared_error(y_valid, preds, squared=False))

In [ ]:
predictions = my_pipeline.predict(test_data)

# Save the predictions to a CSV file
output = pd.DataFrame({'Id': test_data.index,
                       'claim': predictions})
output.to_csv('submission.csv', index=False)